In [0]:
service_credential = dbutils.secrets.get(scope = "databricks-service-principal-secret-scope", key = "Databricks-server-principal-secret")
spark.conf.set("fs.azure.account.auth.type.jcy203datalake.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.jcy203datalake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.jcy203datalake.dfs.core.windows.net", "dc03afd6-fec8-4dfb-ae3a-73a0af445694")
spark.conf.set("fs.azure.account.oauth2.client.secret.jcy203datalake.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.jcy203datalake.dfs.core.windows.net", "https://login.microsoftonline.com/a3b377f7-39e7-4ca3-bf66-ea94bbdcdbf3/oauth2/token")

In [0]:
%python
from datetime import datetime
from pyspark.sql.functions import col, round

# Obtener archivos de curated.
today = datetime.today().strftime("%Y%m%d")
curated_path = f"abfss://currency@jcy203datalake.dfs.core.windows.net/curated"
all_currency_files_df = spark.read.option("header", False).csv(f"{curated_path}/*/{today}/part-*.csv")

# Transformación
all_currency_files_df = all_currency_files_df.withColumnRenamed("_c0", "currency").withColumnRenamed("_c1", "rate")
all_currecy_files_rounded_df = all_currency_files_df.withColumn("rate", round(col("rate").cast("double"), 2))

# No necesito usar distint porque se que el json no tiene monedas duplicadas.
distinct_currencies = []
for row in all_currecy_files_rounded_df.select("currency").collect():
    distinct_currencies.append(row["currency"])

# Paso a Curated
for currency in distinct_currencies:
    all_currecy_files_rounded_df.filter(col('Currency') == currency).write.mode("overwrite").csv(f"abfss://currency@jcy203datalake.dfs.core.windows.net/common/{currency}/{today}")